# denoising auto encoder - DSA feature

- 잡음 제거 오토 인코더 
입력 데이터에 일부 잡음을 추가한 후, 이를 복원하는 과정을 통해 보다 견고한 데이터 표현을 학습하는 신경망 모델

- 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

In [15]:
file_path = "C:/Users/Administrator/dataset/DSA_features.csv"
dsa_data = pd.read_csv(file_path)

# Display the first few rows of the data to understand its structure
dsa_data.head()

,T_xacc_mean,T_xacc_max,T_xacc_min,T_xacc_var,T_xacc_std,T_xacc_skew,T_yacc_mean,T_yacc_max,T_yacc_min,T_yacc_var,...,LL_ymag_std,LL_ymag_skew,LL_zmag_mean,LL_zmag_max,LL_zmag_min,LL_zmag_var,LL_zmag_std,LL_zmag_skew,activity,people
0,7.975714,8.1605,7.6823,0.014395,0.119981,-0.023319,1.083150,1.1832,0.99744,0.002208,...,0.000792,0.177075,-0.057119,-0.054963,-0.059241,6.778722e-07,0.000823,0.036729,sitting,p1
1,7.978250,8.1763,7.8472,0.007551,0.086896,0.552416,1.140865,1.2129,1.05810,0.000784,...,0.000860,-0.286918,-0.057268,-0.054945,-0.059589,7.032302e-07,0.000839,0.347471,sitting,p1
2,7.970894,8.0860,7.8470,0.003092,0.055603,0.100538,1.140962,1.2128,1.07960,0.000508,...,0.000762,-0.134430,-0.057068,-0.054711,-0.059065,6.268222e-07,0.000792,0.045579,sitting,p1
3,7.938412,8.1083,7.6901,0.003763,0.061343,-0.231914,1.165260,1.3170,1.07870,0.002173,...,0.000735,0.021485,-0.056422,-0.053670,-0.058310,8.011245e-07,0.000895,0.240690,sitting,p1
4,7.908930,8.1305,7.8322,0.001741,0.041731,2.042285,1.187504,1.2574,1.09450,0.000662,...,0.000824,-0.148229,-0.055801,-0.053313,-0.057815,6.853423e-07,0.000828,0.258429,sitting,p1


In [4]:
df.shape

(9120, 272)

In [5]:
# 결측치 확인

df.isnull().values.any()

False

In [22]:
# Let's first filter the data for 'normal' activity entries
normal_data = dsa_data[dsa_data['activity'] == 'elevator'].drop(columns=['activity', 'people'])

# Add random noise to the normal data for creating a noisy version
import numpy as np

# Adding Gaussian noise to the data
noise_factor = 0.5
noisy_data = normal_data + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=normal_data.shape)

# Normalize the data to keep values between 0 and 1
normal_data = (normal_data - normal_data.min()) / (normal_data.max() - normal_data.min())
noisy_data = (noisy_data - noisy_data.min()) / (noisy_data.max() - noisy_data.min())

# Check the shape of the data for building the autoencoder
normal_data.shape, noisy_data.shape


((0, 270), (0, 270))

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the structure of the Denoising Autoencoder
input_dim = normal_data.shape[1]

# Encoder
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(128, activation='relu')(input_layer)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)

# Decoder
decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

# Build the autoencoder model
autoencoder = models.Model(input_layer, decoded)

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Train the model
history = autoencoder.fit(
    noisy_data, normal_data, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    verbose=0
)

# Training complete, now we can visualize the results
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 270)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          34,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 270)                 │          34,830 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 270,860 (1.03 MB)

 Trainable params: 90,286 (352.68 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 180,574 (705.37 KB)

In [23]:
# Let's first filter the data for 'standing' and 'elevater' activity entries
standing_elevater_data = dsa_data[dsa_data['activity'].isin(['standing', 'elevater'])].drop(columns=['activity', 'people'])

# Normalize the data
standing_elevater_data = (standing_elevater_data - standing_elevater_data.min()) / (standing_elevater_data.max() - standing_elevater_data.min())

# Check the shape of the filtered data
standing_elevater_data.shape


(480, 270)

In [28]:
# 원본 데이터 복사
No_noise_df = df_total.copy()

# 잡음 추가 비율 설정
noise_factor = 0.5

# 'feature1'에 Gaussian noise 추가
df_total['activity'] = df_total['activity'] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=len(df_total))

# 데이터 확인
df_total.head()


NameError: name 'df_total' is not defined

In [27]:
# 원본 데이터에서 필터링 수행
standing_elevater_data = df_total[df_total['activity'].isin(['standing', 'elevater'])].drop(columns=['activity', 'people'])


NameError: name 'df_total' is not defined

In [ ]:
머신러닝 그거 전에 햇던 거 pca, cnn 해라규